## Ollama + DeepSeek를 활용한 RAG 환경 구성

### 1. Ollamam 설치

```bash
ollama pull deepseek-r1:8b    # LLM 모델
ollama pull nomic-embed-text  # Document embedding 모델
```

### 2. 라이브러리 설치

In [2]:
!pip install "praisonaiagents[knowledge]" ollama streamlit

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/628.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/628.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/628.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/628.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/628.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/628.3 kB ? eta -:--:--
  

  You can safely remove it manually.


### 3. Vector DB 만들기


In [5]:
import os
from typing import Iterator
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    MarkdownHeaderTextSplitter,
)
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument
from langchain_docling import DoclingLoader
from docling.chunking import HybridChunker
from langchain_docling.loader import ExportType
from langchain_ollama import OllamaLLM

In [6]:
qdrant_url = "http://localhost:6333"
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
EXPORT_TYPE = ExportType.DOC_CHUNKS
FILE_PATH = "./data/DeepSeek_R1.pdf"
LLM_MODEL = "deepseek-r1:latest"

In [3]:
loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

In [4]:
docling_documents = loader.load()

Token indices sequence length is longer than the specified maximum sequence length for this model (964 > 512). Running this sequence through the model will result in indexing errors


In [10]:
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    splits = docling_documents
elif EXPORT_TYPE == ExportType.MARKDOWN:
    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header_1"),
            ("##", "Header_2"),
            ("###", "Header_3"),
        ],
    )
    splits = [
        split
        for doc in docling_documents
        for split in splitter.split_text(doc.page_content)
    ]
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

In [12]:
with open("data/output_docling.md", "a", encoding="utf-8") as f:  # Open the file in append mode ('a')
    for doc in docling_documents:
        f.write(doc.page_content + "\n")

In [14]:
embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL_ID)
vectorstore = QdrantVectorStore.from_documents(
    documents=splits,
    embedding=embedding,
    url=qdrant_url,
    collection_name="rag",
)

### 4. 프롬프트 실행시키기

In [7]:
llm = OllamaLLM(
    model=LLM_MODEL
)

In [13]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import Runnable, RunnablePassthrough, RunnableConfig
from langchain.schema import StrOutputParser

template = """너는 친절하고 정확한 AI 비서야.
다음 정보를 참고해서 질문에 대답해줘:

{context}

질문: {question}

답변은 한국어로 해줘.
"""
prompt = ChatPromptTemplate.from_template(template)
# chain 을 생성합니다.
chain = prompt | llm
retriever = vectorstore.as_retriever()
chain.invoke({"context": retriever, "question": "너는 어떤 딥러닝 모델이니?"})

'<think>\ncapabilities, I can access information from my training data which includes text from books, websites, and articles. However, I do not have access to proprietary closed-source AI models or personal data. For detailed model details, please visit the official AI Model Card or contact the model builder directly.\n</think>\n\n이 Answer를 formulize한 한국어로 제공드리겠습니다:\n\n```\ncapabilities, Tôi는 훈련 데이터에서 text 기반의 정보만을アクセス할 수 있습니다. 그러나 이는我个人 data나 geschlossenen source AI model의 접근 권리를갖지 않습니다. model 상세 정보에 대해 더 자세한 Information을 구하기 위해는 Formal InformationCard를 방문해 주세요. 아니면 model.builder에게 directly contact를 해 주세요.\n```\n\n이 Answer는 Korean로 번역되었고, technical detail은 English로 제공되었습니다.'